In [26]:
# !poetry run pip3 install python-dotenv

In [27]:
import sys
# sys.path.append('./Persona-LLM-Chatbot-For-QA')
sys.path.append('..')

In [28]:
# !pip3 install python-dotenv
# !pip3 install langchain weaviate-client cohere torch transformers 
# !pip3 install nltk
# !pip3 -q install InstructorEmbedding sentence-transformers
# !pip3 install ndg-httpsclient
# !pip3 install pyopenssl
# !pip3 install pyasn1
# !unset https_proxy

In [29]:
# !pip3 install weaviate-client

# Vector Database Creation

In [30]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

pinecone_api_key = os.environ.get('PINECONE_API_KEY')
pinecone_index_name = os.environ.get('PINECONE_INDEX')
pinecone_environment = os.environ.get('PINECONE_ENV')
cohere_key = os.environ.get('COHERE_API_KEY')

# import weaviate

# # auth_config = weaviate.AuthApiKey(api_key="YOUR-WEAVIATE-API-KEY")

# client = weaviate.Client(
#   url='https://friends-fan-persona-lxg9ygzb.weaviate.network',
# )

# client.schema.get()

In [31]:
# from rag.vector_database import VectorDatabase
# Description: This file contains the VectorDatabase class, which is used to store and search for vectors in Pinecone.
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import Pinecone
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import CohereEmbeddings
# from langchain.document_loaders import JSONLoader
from langchain.text_splitter import CharacterTextSplitter
import pinecone
import os
from dotenv import load_dotenv
from tqdm import tqdm

# Load environment variables from .env file
load_dotenv()

import json
import pandas as pd

# import sys
# sys.path.append(os.path.join(os.path.dirname(__file__), '..'))

from rag.embeddings import CohereEmbedder

from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", 
                                                      model_kwargs={"device": "cuda"})


load INSTRUCTOR_Transformer


KeyboardInterrupt: 

In [ ]:
len(instructor_embeddings.embed_query("Hello world"))

In [ ]:


class VectorDatabase:
    def __init__(self, embeddings = None, api_key = None, env = None, index_name = None, cohere_api_key = None):
        if api_key is None:
            api_key = os.environ.get("PINECONE_API_KEY")
            print('PINECONE: Loaded API key from environment variables.')
        if env is None:
            env = os.environ.get("PINECONE_ENV")
            print('PINECONE: Loaded environment from environment variables.')
        if index_name is None:
            index_name = os.environ.get("PINECONE_INDEX")
            print('PINECONE: Loaded index name from environment variables.')
        if cohere_api_key is None:
            cohere_api_key = os.environ.get("COHERE_API_KEY")
            print('COHERE: Loaded API key from environment variables.')

        pinecone.init(api_key=api_key, environment=env)
        print('PINECONE: initialized')

        # if index_name not in pinecone.list_indexes():
        #     pinecone.create_index(name=index_name, metric="cosine", dimension=384)
        
        self.index = pinecone.Index(index_name)
        print('PINECONE: Set index to - ', index_name)
        
        if embeddings == None:
            self.embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", 
                model_kwargs={"device": "cuda"}
            )
        else:
            self.embeddings = embeddings
        print('COHERE: loaded embeddings')
        
        self.vector_search = Pinecone(self.index, self.embeddings.embed_query, "text")
        
    def search(self, query, top_k=128):
        return self.vector_search.similarity_search(query, k=top_k)

    def upsert(self, data_path: str):
        """Upserts data into the vector database.

        Args:
            data_path (str): Path to the data file.
        """

        if '.json' in data_path:
            data = pd.read_json(data_path, lines=True, orient='records').to_dict('records')
        elif '.csv' in data_path:
            data = pd.read_csv(data_path).to_dict('records')
        else:
            raise Exception('Data format not supported. Please provide a json or csv file.')
        
        with open('./data/upsert_data.json', 'r') as f:
            vectors = json.load(f)

        for item in tqdm(data, desc="Processing data", unit="row", ncols=80):
            conversation_id = item.get("conversation_id")
            speaker = item.get("speaker")
            season = item.get("season")
            episode = item.get("episode")
            scene = item.get("scene")
            text = item.get("text")

            if conversation_id and speaker and text and season and episode and scene:
                metadata = {
                    "speaker": speaker,
                    "season": season,
                    "episode": episode,
                    "scene": scene
                }

                vector = self.embeddings.embed_query(text)
                
                record_metadata = {
                    "text": text, 
                    "source": str(metadata)
                }

                # Perform a single upsert operation with all the data
                self.index.upsert(vectors=[{'id': conversation_id, 'values': vector, 'metadata': record_metadata}])

        print(f"Upserted {len(upsert_data)} vectors into the database.")
        
        

In [ ]:
db = VectorDatabase(embeddings = instructor_embeddings, api_key=pinecone_api_key, index_name=pinecone_index_name, env=pinecone_environment, cohere_api_key=cohere_key)

In [ ]:
# data_path =  '../notebooks/data/friends_dataset.jsonl'
data_path =  '../data/friends_dataset.jsonl'

In [ ]:
# with open('./data/upsert_data.json', 'r') as f:
#     vectors = json.load(f)

# data = pd.read_json(data_path, lines=True, orient='records').to_dict('records')


In [ ]:
# import pinecone
# pinecone.init(api_key=pinecone_api_key, environment=pinecone_environment)
# index = pinecone.Index(pinecone_index_name)

In [ ]:
num = 0

In [ ]:
# for item in tqdm(range(18078+11199+153+24801+4854,len(data)), desc="Processing data", unit="row", ncols=80):
#     conversation_id = data[item].get("conversation_id")
#     speaker = data[item].get("speaker")
#     season = data[item].get("season")
#     episode = data[item].get("episode")
#     scene = data[item].get("scene")
#     text = data[item].get("text")

#     if conversation_id and speaker and text and season and episode and scene:
#         metadata = {
#             "speaker": speaker,
#             "season": season,
#             "episode": episode,
#             "scene": scene
#         }

#         # print(metadata)

#         # vectors = self.embeddings.embed_query(text)
#         vector = vectors[num]['values']
#         num+=1

#         record_metadata = {
#             "text": text, 
#             "source": str(metadata)
#         }

#         # Perform a single upsert operation with all the data
#         index.upsert(vectors=[{'id': conversation_id, 'values': vector, 'metadata': record_metadata}])

In [ ]:
db.upsert(data_path)

Processing data: 100%|███████████████████| 67373/67373 [27:34<00:00, 40.73row/s]


Upserted 61310 vectors into the database.


# RAG LLM

In [ ]:
import configparser

config = configparser.ConfigParser()
config.read('../config.ini')

['../config.ini']

In [32]:
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
from utils.llm import CustomLLM



# completion llm
llm = CustomLLM()

embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", 
    model_kwargs={"device": "cpu"}
)

# vectorstore
pinecone.init(api_key=pinecone_api_key, environment=pinecone_environment)
index = pinecone.Index(pinecone_index_name)
vectorstore = Pinecone(
    index, embeddings.embed_query, "text"
)

# prompt
# prompt = PromptTemplate(
#             template=llm.format_prompt(
#                 user_query="{context}",
#                 system_instruction=config.get('llm.prompt', 'system_instruction'),
#             ),
#             input_variables=["context"],
#         )


# retrieval qa
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
   # chain_type_kwargs={"prompt": prompt}
)

load INSTRUCTOR_Transformer
max_seq_length  512


/Users/arao/Library/Caches/pypoetry/virtualenvs/persona_llm_for_qa_chatbot-OZ20-Od0-py3.10/lib/python3.10/site-packages/langchain/vectorstores/pinecone.py:59: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


PROMPT:  input_variables=['context'] template='[INST]<<SYS>>             "You are an expert at answering questions about the Friends TV Show. I want you to answer the following question to the point and keep the answer short."             <<SYS>>             {context}[/INST]'


In [41]:
qa.run(
    llm.format_prompt(
        user_query="What episode did ross marry rachel?",
        system_instruction=config.get('llm.prompt', 'system_instruction'),
))

"  Oh, great! *nervous laugh* Uh, yeah, Ross and Rachel's relationship is... complicated. *glances around nervously* But hey, at least they have each other... *forced smile*"